In [1]:
from IPython.display import display, Math
from functools import reduce
import numpy as np
import matplotlib
import matplotlib.pyplot as plt


In [2]:
def add(x, y): return ['+', x, y]
def sub(x, y): return ['-', x, y]
def mul(x, y): return ['*', x, y]
def div(x, y): return ['/', x, y]


def power(x, y): return ['^', x, y]


def sqrt_(x): return ['sqrt', x]
def ln_(x): return ['ln', x]
def exp_(x): return ['exp', x]


def negative(x): return ['~', x]


def sin_(x): return ['sin', x]
def cos_(x): return ['cos', x]
def tg_(x): return ['tg', x]
def ctg_(x): return ['ctg', x]


def arcsin_(x): return ['arcsin', x]
def arccos_(x): return ['arccos', x]
def arctg_(x): return ['arctg', x]
def arcctg_(x): return ['arcctg', x]


def sh_(x): return ['sh', x]
def ch_(x): return ['ch', x]
def th_(x): return ['th', x]
def cth_(x): return ['cth', x]


def arsh_(x): return ['arsh', x]
def arch_(x): return ['arch', x]
def arth_(x): return ['arth', x]
def arcth_(x): return ['arcth', x]


def fact_(x): return ['fact', x]

# Операторы для сложения и умножения нескольких членов сразу


def Sum(*x): return ['S', *x]
def Product(*x): return ['P', *x]


In [3]:
def fix(x):
    if isinstance(x, Expression):
        return x
    return Expression(x)

# Корень, логарифм, экспонента


def sqrt(x):
    return Expression(sqrt_(fix(x).T))


def ln(x):
    return Expression(ln_(fix(x).T))


def exp(x):
    return Expression(exp_(fix(x).T))

# Тригонометрические функции


def sin(x):
    return Expression(sin_(fix(x).T))


def cos(x):
    return Expression(cos_(fix(x).T))


def tg(x):
    return Expression(tg_(fix(x).T))


def ctg(x):
    return Expression(ctg_(fix(x).T))

# Обратные тригонометрические функции


def arcsin(x):
    return Expression(arcsin_(fix(x).T))


def arccos(x):
    return Expression(arccos_(fix(x).T))


def arctg(x):
    return Expression(arctg_(fix(x).T))


def arcctg(x):
    return Expression(arcctg_(fix(x).T))

# Гиперболические функции — пункт 2.6


def sh(x):
    return Expression(sh_(fix(x).T))


def ch(x):
    return Expression(ch_(fix(x).T))


def th(x):
    return Expression(th_(fix(x).T))


def cth(x):
    return Expression(cth_(fix(x).T))

# Обратные гиперболические функции — пункт 2.7


def arsh(x):
    return Expression(arsh_(fix(x).T))


def arch(x):
    return Expression(arch_(fix(x).T))


def arth(x):
    return Expression(arth_(fix(x).T))


def arcth(x):
    return Expression(arcth_(fix(x).T))

# Факториал — пункт 7


def fact(x):
    return Expression(fact_(fix(x).T))


**Класс Expression**

In [4]:
class Expression:
    def __init__(self, x):
        self.T = x

    # Арифметически операции для левого и правого аргументов
    def __add__(self, x):
        return Expression(add(self.T, fix(x).T))

    def __radd__(self, x):
        return Expression(add(fix(x).T, self.T))

    def __sub__(self, x):
        return Expression(sub(self.T, fix(x).T))

    def __rsub__(self, x):
        return Expression(sub(fix(x).T, self.T))

    def __mul__(self, x):
        return Expression(mul(self.T, fix(x).T))

    def __rmul__(self, x):
        return Expression(mul(fix(x).T, self.T))

    def __truediv__(self, x):
        return Expression(div(self.T, fix(x).T))

    def __rtruediv__(self, x):
        return Expression(div(fix(x).T, self.T))

    # Возведение в степень для левого и правого аргументов
    def __pow__(self, x):
        return Expression(power(self.T, fix(x).T))

    def __rpow__(self, x):
        return Expression(power(fix(x).T, self.T))

    # Унарный минус
    def __neg__(self):
        return Expression(negative(self.T))

    def __getitem__(self, k):
        if self.T[0] == 'enum':
            return Expression(simplify(subs(self.T, x, k)[1]))
        return Expression(self.T[k + 1])

    def __len__(self):
        if isinstance(self.T, list):
            return len(self.T) - 1
        return 0


def show(x):
    display(Math(toLatex(x)))


**Класс Rational**

In [5]:
def Rational(n, m):
    if n == 0:
        return ['q', 0, 1]
    if m < 0:
        n, m = -n, -m
    d = GCD(abs(n), m)
    return ['q', n // d, m // d]

# Приведение к объекту класса Expression


def Q(n, m):
    return Expression(Rational(n, m))

# Greatest Common Divisor


def GCD(n, m):
    while m:
        n, m = m, n % m
    return n


In [6]:
def enclose(x):
    return f'\\left( {x} \\right)'


def toLatex(x):
    if isinstance(x, Expression):
        return toLatex(x.T)

    if isinstance(x, (int, str)):
        return str(x)

    op = x[0]
    if op in '+-*/^':
        l, r = toLatex(x[1]), toLatex(x[2])
        if op == '+':
            return f'{l} + {r}'
        if op == '-':
            return f'{l} - {r}'
        if op == '*':
            if isinstance(x[1], list) and x[1][0] in '+-' or l[0] == '-':
                l = enclose(l)
            if isinstance(x[2], list) and x[2][0] in '+-' or r[0] == '-':
                r = enclose(r)
            return f'{l} \\cdot {r}'
        if op == '/':
            return f'\\dfrac{{{l}}}{{{r}}}'
        if op == '^':
            if isinstance(x[1], list):
                l = enclose(l)
            return f'{{{l}}}^{{{r}}}'
    if op == '=':
        return f'{toLatex(x[1])} = {toLatex(x[2])}'
    if op == 'S':
        A = [toLatex(a) for a in x[1:]]
        for k in range(len(A)):
            if A[k][0] == '-':
                A[k] = enclose(A[k])
            return ' + '.join(A)
    if op == 'P':
        A = [toLatex(a) for a in x[1:]]
        for k in range(1, len(x)):
            if isinstance(x[k], list) and x[k][0] == 'S' or A[k-1][0] == '-':
                A[k-1] = enclose(A[k-1])
        return ' \\cdot '.join(A)
    if op == 'q':
        if x[2] == 1:
            return str(x[1])
        if x[1] > 0:
            return f'\\dfrac{{{x[1]}}}{{{x[2]}}}'
        return f'-\\dfrac{{{-x[1]}}}{{{x[2]}}}'
    if op == 'set':
        A = [toLatex(a) for a in x[1:]]
        return f'\\left\\{{ {",".join(A)} \\right\\}}'
    else:
        a = enclose(toLatex(x[1]))
        if op == '~':
            return f'- {a}'
        # Корень, логарифм, экспонента
        if op == 'sqrt':
            return f'\\sqrt{{{a}}}'
        if op == 'ln':
            return f'\\ln {a}'
        if op == 'exp':
            return f'e ^ {{{a}}}'

        # Тригонометрические функции
        if op == 'sin':
            return f'\\sin {a}'
        if op == 'cos':
            return f'\\cos {a}'
        if op == 'tg':
            return f'\\tg {a}'
        if op == 'ctg':
            return f'\\ctg {a}'

        # Обратные тригонометрические функции
        if op == 'arcsin':
            return f'\\arcsin {a}'
        if op == 'arccos':
            return f'\\arccos {a}'
        if op == 'arctg':
            return f'\\arctg {a}'
        if op == 'arcctg':
            return f'\\arcctg {a}'

        # Гиперболические функции — пункт 2.6
        if op == 'sh':
            return f'\\sh {a}'
        if op == 'ch':
            return f'\\ch {a}'
        if op == 'th':
            return f'\\th {a}'
        if op == 'cth':
            return f'\\cth {a}'

        # Обратные гиперболические функции — пункт 2.7
        if op == 'arsh':
            return f'\\arsh {a}'
        if op == 'arch':
            return f'\\arch {a}'
        if op == 'arth':
            return f'\\arth {a}'
        if op == 'arcth':
            return f'\\arcth {a}'

        if op == 'fact':
            return f'{a}!'


In [7]:
def symbols(s):
    S = s.split()
    if len(S) == 1:
        return Expression(s)
    return (Expression(i) for i in S)


def getSymbols(s, symb_arr=[]):  # Пункт 4
    if isinstance(s, list):
        for i in range(1, len(s)):
            if isinstance(s[i], int):
                pass
            elif isinstance(s[i], list):
                getSymbols(s[i], symb_arr)
            else:
                if s[i] not in symb_arr:
                    symb_arr.append(s[i])
    elif isinstance(s, Expression):
        for i in range(1, len(s.T)):
            if isinstance(s.T[i], int):
                pass
            elif isinstance(s.T[i], list):
                getSymbols(s.T[i], symb_arr)
            else:
                if s.T[i] not in symb_arr:
                    symb_arr.append(s.T[i])
    return symb_arr


def isConst(x, v=[]):  # Пункт 5
    if not v:
        if not getSymbols(x, []):
            return True
        return False
    else:
        if getSymbols(x, []) == v:
            return False
        return True


In [8]:
def evalf(x, arg={}):
    if isinstance(x, Expression):
        if isinstance(arg, dict):
            arg = {i.T: arg[i] for i in arg}
        else:
            arg = {'$': arg}
        return evalf(x.T, arg)

    if isinstance(x, int):
        return x

    if isinstance(x, str):
        if x in arg:
            return arg[x]
        return arg['$']

    op = x[0]
    if op in '+-*/^':
        l, r = evalf(x[1], arg), evalf(x[2], arg)
        if op == '+':
            return l + r
        if op == '-':
            return l - r
        if op == '*':
            return l * r
        if op == '/':
            return l / r
        if op == '^':
            return l ** r
    else:
        a = evalf(x[1], arg)
        if op == '~':
            return -a

        # Корень, логарифм, экспонента
        if op == 'sqrt':
            return np.sqrt(a)
        if op == 'ln':
            return np.log(a)
        if op == 'exp':
            return np.exp(a)

        # Тригонометрические функции
        if op == 'sin':
            return np.sin(a)
        if op == 'cos':
            return np.cos(a)
        if op == 'tg':
            return np.tan(a)
        if op == 'ctg':
            return np.cot(a)

        # Обратные тригонометрические функции
        if op == 'arcsin':
            return np.arcsin(a)
        if op == 'arccos':
            return np.arccos(a)
        if op == 'arctg':
            return np.arctan(a)
        # if op == 'arcctg':
        #  return np.arccot(a)

        # Обратные тригонометрические функции
        if op == 'sh':
            return np.sinh(a)
        if op == 'ch':
            return np.cosh(a)
        if op == 'th':
            return np.tanh(a)
        # if op == 'cth':
        #  return np.coth(a)

        # Обратные тригонометрические функции
        if op == 'arsh':
            return np.arcsinh(a)
        if op == 'arch':
            return np.arccosh(a)
        if op == 'arth':
            return np.arctanh(a)
        # if op == 'arcth':
        #  return np.arccoth(a)

        # Факториал
        if op == 'fact':
            if a <= 1:
                return 1
            elif a == 2:
                return 2
            elif a > 2:
                return a * evalf(fact(a - 1))
            return None
        if op in 'SP':
            A = [evalf(a, arg) for a in x[1:]]
            if op == 'S':
                return sum(A)
            if op == 'P':
                return reduce(lambda x, y: x * y, A, 1)
    return None


**Однократная производная**

In [9]:
def Z(x):
    return Expression(x)


def isInt(x):
    return isinstance(x.T, int)


def diff(y, x):
    if isInt(y):
        return Z(0)

    if isinstance(y.T, str):
        if y.T == x.T:
            return Z(1)
        return Z(0)

    op = y.T[0]
    if op in '+-*/^':
        u, v = Expression(y.T[1]), Expression(y.T[2])
        du, dv = diff(u, x), diff(v, x)
        if op == '+':
            return du + dv
        if op == '-':
            return du - dv
        if op == '*':
            if isInt(u):
                return u * dv
            if isInt(v):
                return du * v
            return du * v + u * dv
        if op == '/':
            if isInt(u):
                return - u * dv / v ** 2
            if isInt(v):
                return du / v
            return (du * v - u * dv) / v ** 2
        if op == '^':
            if isInt(v):  # u(x) ^ n
                n = v.T
                if n == 0:
                    return Z(0)
                if n == 1:
                    return du
                if du.T == 1:
                    return n * u ** (n - 1)
                return n * u ** (n - 1) * du
            if isInt(u):  # n ^ v(x)
                n = u.T
                return ln(n) * (u ** v) * dv
            return u ** v * (ln(u) + dv)  # u(x) ^ v(x)
    else:
        u = Expression(y.T[1])
        du = diff(u, x)
        if op == '~':
            return -du

        # Корень, логарифм, экспонента
        if op == 'sqrt':
            return du / (2 * sqrt(u))
        if op == 'ln':
            return du / u
        if op == 'exp':
            return exp(u) * du

        # Тригонометрические функции
        if op == 'sin':
            return cos(u) * du
        if op == 'cos':
            return -sin(u) * du
        if op == 'tg':
            return du / cos(u) ** 2
        if op == 'ctg':
            return -du / sin(u) ** 2

        # Обратные тригонометриские функции
        if op == 'arcsin':
            return du / sqrt(1 - u ** 2)
        if op == 'arccos':
            return -du / sqrt(1 - u ** 2)
        if op == 'arctg':
            return du / (1 + u ** 2)
        if op == 'arcctg':
            return -du / (1 + u ** 2)

        # Гиперболические функции
        if op == 'sh':
            return ch(u) * du
        if op == 'ch':
            return sh(u) * du
        if op == 'th':
            return du / ch(u) ** 2
        if op == 'cth':
            return -du / sh(u) ** 2

        # Обратные гиперболические функции
        if op == 'arsh':
            return du / sqrt(x ** 2 + 1)
        if op == 'arch':
            return du / sqrt(x ** 2 - 1)
        if op == 'arth':
            return du / (1 - x ** 2)
        if op == 'arcth':
            return du / (1 - x ** 2)
    return None


**Многократная производная**

In [10]:
def Diff(y, x, n=1):
    dy = y
    for i in range(n):
        dy = diff(dy, x)
    return dy


**Нормализация (Задание 5)**

In [11]:
def normalize(x):
    if isinstance(x, (int, str)):
        return x
    op, arg = x[0], x[1:]
    A = [normalize(a) for a in arg]
    if op in '+S':  # сборка сумм
        R = []
        for a in A:
            R += a[1:] if isinstance(a, list) and a[0] in '+S' else [a]
        return Sum(*R)
    if op in '*P':  # сборка произведений
        R = []
        for a in A:
            R += a[1:] if isinstance(a, list) and a[0] in '*P' else [a]
        return Product(*R)
    if op == '~':  # унарный минус
        return normalize(Product(-1, A[0]))
    if op == '-':  # бинарный минус
        return normalize(Sum(A[0], Product(-1, A[1])))
    if op == '/':  # деление
        return normalize(Product(A[0], power(A[1], -1)))
    if op == '^':  # нормализация степеней
        if isinstance(A[0], list) and A[0][0] == 'P':
            B = [power(a, A[1]) for a in A[0][1:]]
            return normalize(Product(*B))
        if isinstance(A[0], list) and A[0][0] == '^':
            d = Product(A[0][2], A[1])
            return normalize(power(A[0][1], d))
        if isinstance(A[0], list) and A[0][0] == 'sqrt':
            if A[1] % 2 == 0:
                return normalize(power(A[0][1], Rational(A[1] // 2, 1)))
    if op == 'sqrt':
        if isinstance(A[0], list) and A[0][0] == '^' and A[0][2] % 2 == 0:
            return normalize(power(A[0][1], Rational(A[0][2] // 2, 1)))
    return x


**Сортировка списочных выражений (Задание 5)**

In [12]:
def sortx(x):
    if isinstance(x, list):
        op, arg = x[0], x[1:]
        A = [sortx(a) for a in arg]
        if op in 'SP':
            A.sort(key=lambda a: str(a))
        return [op, *A]
    return x


**Упрощение (Задание 5)**

In [13]:
def simplify(x):
    if isinstance(x, Expression):
        return Expression(simplify(x.T))
    if isinstance(x, str):
        return x
    if isinstance(x, (int, float)):
        return Rational(x, 1)
    if x[0] == 'q':
        return x
    x = sortx(normalize(x))
    op, arg = x[0], x[1:]
    A = [simplify(a) for a in arg]
    if op == '^':  # Упрощение степеней
        if isinstance(A[1], list) and A[1][0] == 'q' and A[1][2] == 1:
            if isinstance(A[0], list) and A[0][0] == 'q':
                n, m, d = A[0][1], A[0][2], A[1][1]
                if d >= 0:
                    return Rational(n ** d, m ** d)
                else:
                    return Rational(m ** (-d), n ** (-d))
        if A[1][1] == 1:  # Пункт 3 ['переменная', ['q', числитель, знаменатель]]
            return A[0]
        if A[1][1] == 0:  # Пункт 3
            return Rational(1, 1)
        if A[0][0] == 'q' and A[0][1] == 1:  # Пункт 3
            return Rational(1, 1)
    if op == 'P':  # Упрощение произведений
        # if A[1][0] == 'q' and A[0][0] == 'S': # Пункт 9
        n, m, V = 1, 1, []
        for a in A:
            if isinstance(a, list) and a[0] == 'q':
                n, m = n * a[1], m * a[2]
            else:
                V.append(a)
        q = Rational(n, m)
        if len(V) > 1:
            V = collectSameTermsP(V)
        if q[1] == 0 or V == []:
            return q
        if q[1] == 1 and q[2] == 1 and len(V) == 1:
            return V[0]
        A = [q, *V] if q[1] != 1 or q[2] != 1 else [*V]
    if op == 'S':  # Упрощение сумм
        n, m, V = 0, 1, []
        for a in A:
            if isinstance(a, list) and a[0] == 'q':
                n, m = n * a[2] + m * a[1], m * a[2]
            else:
                V.append(a)
        q = Rational(n, m)
        if len(V) > 1:
            V = collectSameTerms(V)
        if V == []:
            return q
        if q[1] == 0 and len(V) == 1:
            return V[0]
        A = [q, *V] if q[1] != 0 else [*V]
    # if op == 'sqrt':
    if op == 'ln':
        if A[0][0] == 'q' and A[0][1] == 1:
            return Rational(0, 1)
    if op == 'exp':
        if A[0][0] == 'ln':
            return A[0][1]
        if A[0][0] == 'q' and A[0][1] == 0:
            return Rational(1, 1)
    if op == 'sin':
        if A[0][0] == 'arcsin':
            return A[0][1]
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 180) == 0:
            return Rational(0, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 360) == 30 or ((A[0][1] / A[0][2]) % 360) == 150:
            return Rational(1, 2)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 360) == 90:
            return Rational(1, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 360) == 210 or ((A[0][1] / A[0][2]) % 360) == 330:
            return Rational(-1, 2)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 360) == 270:
            return Rational(-1, 1)
    if op == 'arcsin':
        if A[0][0] == 'sin':
            return A[0][1]
    if op == 'cos':
        if A[0][0] == 'arccos':
            return A[0][1]
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 360) == 0:
            return Rational(1, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 360) == 60 or ((A[0][1] / A[0][2]) % 360) == 300:
            return Rational(1, 2)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 180) == 90:
            return Rational(0, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 360) == 180:
            return Rational(-1, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 360) == 120 or ((A[0][1] / A[0][2]) % 360) == 240:
            return Rational(-1, 2)
    if op == 'arccos':
        if A[0][0] == 'cos':
            return A[0][1]
    if op == 'tg':
        if A[0][0] == 'arctg':
            return A[0][1]
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 180) == 0:
            return Rational(0, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 180) == 45:
            return Rational(1, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 180) == 135:
            return Rational(-1, 1)
    if op == 'arctg':
        if A[0][0] == 'tg':
            return A[0][1]
    if op == 'ctg':
        if A[0][0] == 'arcctg':
            return A[0][1]
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 180) == 45:
            return Rational(1, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 180) == 90:
            return Rational(0, 1)
        if A[0][0] == 'q' and ((A[0][1] / A[0][2]) % 180) == 135:
            return Rational(-1, 1)
    if op == 'arcctg':
        if A[0][0] == 'ctg':
            return A[0][1]
    return [op, *A]


def collectSameTerms(A):
    count, X = 1, []
    for k in range(1, len(A)):
        if A[k] == A[k - 1]:
            count += 1
            continue
        X.append(simplify(Product(Rational(count, 1), A[k - 1])))
        count = 1
    X.append(simplify(Product(Rational(count, 1), A[-1])))
    return X


def collectSameTermsP(A):
    for k in range(0, len(A)):
        if not isinstance(A[k], list):
            A[k] = ['^'] + [A[k]] + [['q', 1, 1]]
    count, X = A[-1][2][1], []
    for k in range(0, len(A)):
        if A[k][1] == A[k - 1][1]:
            if k == len(A) - 1:
                X.append(A[k - 1][1] ** Expression(Rational(count, 1)))
            count += A[k][2][1]
        elif A[k][1] != A[k - 1][1]:
            X.append(A[k - 1][1] ** Expression(Rational(count, 1)))
            count = A[k][2][1]
    return X


**Решение уравнений (Задание 6)**

In [14]:
def Eq(x, y):
    return Expression(['=', fix(x).T, fix(y).T])


def lhs(f):
    if f.T[0] == '=':
        return Expression(f.T[1])
    return f


def rhs(f):
    if f.T[0] == '=':
        return Expression(f.T[2])
    return f


**Вспомогательные функции (Задание 6)**

In [15]:
def contains(f, x):
    if isinstance(f, Expression):
        return contains(f.T, x.T)
    if isinstance(f, str):
        return f == x
    if isinstance(f, list):
        for a in f[1:]:
            if contains(a, x):
                return True
    return False


def subs(f, x, x0):
    if isinstance(f, Expression):
        return Expression(subs(f.T, fix(x).T, fix(x0).T))
    if isinstance(f, str):
        if f == x:
            return x0
    if isinstance(f, list):
        A = [subs(a, x, x0) for a in f[1:]]
        return [f[0], *A]
    return f


def isSolution(eq, x, x0):
    if classifyEquation(eq, x) == 'Linear':
        for i in solveLinear(eq, x):
            if toLatex(x0) == toLatex(i):
                return True
    elif classifyEquation(eq, x) == 'Unique':
        for i in solveUnique(eq, x):
            if toLatex(x0) == toLatex(i):
                return True
    elif classifyEquation(eq, x) == 'Quadratic':
        for i in solveQuadratic(eq, x):
            if toLatex(x0) == toLatex(i):
                return True
    return False


def classifyEquation(eq, x):
    f = lhs(eq) - rhs(eq)
    df = diff(f, x)
    k = simplify(df)
    if not contains(k, x):
        return 'Linear'
    dff = diff(df, x)
    kk = simplify(dff)
    if not contains(kk, x):
        return 'Quadratic'
    dfff = diff(dff, x)
    # kkk = simplify(dfff)
    # if not contains(kkk, x):
    #  return 'Cubic'
    return 'Unique'


def solve(eq, x):
    if classifyEquation(eq, x) == 'Linear':
        return solveLinear(eq, x)
    elif classifyEquation(eq, x) == 'Quadratic':
        return solveQuadratic(eq, x)
    elif classifyEquation(eq, x) == 'Unique':
        return solveUnique(eq, x)


**Множества (Задание 6)**

In [16]:
# Конечное множество
def Set(*X):
    A = [x for x in X]
    return Expression(['set', *A])

# Перечислимое множество


def EnumSet(*X):  # no.7, getitem in exp class
    A = [x for x in X]
    return Expression(['enum', *A])


**Решение уравнений (Задание 6)**

In [17]:
# Линейное уравнение (Linear)
def solveLinear(eq, x):
    f = lhs(eq) - rhs(eq)
    df = diff(f, x)
    k = simplify(df)
    # if contains(k, x):
    #   raise ValueError("Equation is not linear")
    b = simplify(subs(f, x, Z(0)))
    if k.T != ['q', 0, 1]:
        return Set(simplify(-b / k))
    return Set()

# Уравнение с единственным вхождением переменной (Unique)


def solveUnique(eq, x):
    F = simplify(lhs(eq) - rhs(eq))
    x0 = solveUnique_(F.T, 0, x.T)
    if x0 == None:
        return Set()
    return Set(Expression(x0))


def solveUnique_(L, R, x):
    if L == x:
        return R
    op = L[0]
    arg = L[1:]
    if op == 'S':
        N, X = [], None
        for a in arg:
            if contains(a, x):
                X = a
            else:
                N.append(a)
        R = simplify(sub(R, Sum(*N)))
        return solveUnique_(X, R, x)
    if op == 'P':
        N, X = [], None
        for a in arg:
            if contains(a, x):
                X = a
            else:
                N.append(a)
        R = simplify(div(R, Product(*N)))
        return solveUnique_(X, R, x)
    if op == '^':
        X, b = L[1], L[2]
        R = simplify(power(R, div(1, b)))
        return solveUnique_(X, R, x)
    if op == 'sqrt':
        X, R = L[1], simplify(power(R, 2))
        return solveUnique_(X, R, x)
    if op == 'ln':
        X, R = L[1], simplify(exp_(R))
        return solveUnique_(X, R, x)
    if op == 'exp':
        X, R = L[1], simplify(ln_(R))
        return solveUnique_(X, R, x)

# Квадратное уравнение


def solveQuadratic(eq, x):
    f = lhs(eq)
    a = 1
    if not isinstance(f.T[1][2], list):
        b = 1
    else:
        b = f.T[1][2][1]
    if f.T[1][0] == '-':
        b = -b  # Если перед коэффициентом b стоит знак "минус", то он переносится к коэффициенту
    c = f.T[2]  # Коэффициент c
    if f.T[0] == '-':
        c = -c  # Если перед коэффициентом c стоит знак "минус", то он переносится к коэффициенту
    D = b ** 2 - 4 * a * c
    if D < 0:
        return Set()
    if D == 0:
        x1 = -b / 2
        return Set(simplify(x1))
    if D > 0:
        x1 = (-b + sqrt(D)) / 2
        x2 = (-b - sqrt(D)) / 2
        return Set(simplify(x1), simplify(x2))


In [19]:
x = symbols('x')

print()
print("Решение квадратных уравнений")
eq = Eq(x ** 2 + 4 * x + 4, 0)
show(solveQuadratic(eq, x))

print()
print("isSolution")
eq = Eq(x ** 2 - 3 * x + 2, 0)
show(solve(eq, x))
print(isSolution(eq, x, Q(1, 2) * (3 + sqrt(1))))

print()
print("classifyEquation")
eq = Eq(x ** 2 + 5 * x - 2, 0)
show(eq)
print(classifyEquation(eq, x))

print()
print("solve")
eq = Eq(5 - 2 * x, 0)
show(eq)
show(solve(eq, x))

eq = Eq(x ** 2 - 3 * x + 2, 0)
show(eq)
show(solve(eq, x))

eq = Eq(x ** 2 + 4 * x + 4, 0)
show(eq)
show(solve(eq, x))

eq = Eq(x ** 2 - x + 2, 0)
show(eq)
show(solve(eq, x))

eq = Eq(x ** 2 - 2 * x - 1, 0)
show(eq)
show(solve(eq, x))

print()
print("Перечислимое множество")
es1 = EnumSet(3 * x ** 2 + 1)
show(es1[6])



Решение квадратных уравнений


<IPython.core.display.Math object>


isSolution


<IPython.core.display.Math object>

True

classifyEquation


...

<IPython.core.display.Math object>

<IPython.core.display.Math object>

<IPython.core.display.Math object>


Перечислимое множество


<IPython.core.display.Math object>